In [ ]:
# Install the 'instagrapi' library to interact with Instagram using its private API
!pip install instagrapi

In [ ]:
# Import necessary libraries: Client for Instagram API access, getpass for secure password input
from instagrapi import Client
from getpass import getpass

# Create an instance of the Instagram client
cl = Client()

# Prompt the user to input their Instagram username and password securely
username = input("Enter your Instagram username: ")
password = getpass("Enter your Instagram password: ")

# Try logging in without 2FA
try:
    cl.login(username, password)
    print("Logged in successfully without 2FA")
except Exception as e:
    print("2FA required for login")

    # Prompt for 2FA code if required
    verification_code = input("Enter your 2FA code: ")
    try:
        cl.login(username, password, verification_code=verification_code)
        print("Logged in successfully with 2FA")
    except Exception as e2:
        print("Login failed:", e2)

In [ ]:
import json

# Load followers from JSON file
with open("followers_1.json", "r", encoding="utf-8") as f:
    followers_data = json.load(f)

# Extract follower usernames from the loaded data
followers = {
    entry["string_list_data"][0]["value"]
    for entry in followers_data
    if entry.get("string_list_data")
}

# Load followings from JSON file
with open("following.json", "r", encoding="utf-8") as f:
    following_data = json.load(f)

# Extract following usernames from the loaded data
followings = {
    entry["string_list_data"][0]["value"]
    for entry in following_data["relationships_following"]
    if entry.get("string_list_data")
}

# Identify users you follow but who don't follow you back
not_following_back = followings - followers

# Display the number and list of users who are not following you back
print(f"Number of users not following back: {len(not_following_back)}\n")
for username in sorted(not_following_back):
    print(username)

In [ ]:
import csv

# Save the list of users not following back into a CSV file
with open("not_following_back.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Username"])  # Write header
    for username in sorted(not_following_back):
        writer.writerow([username])  # Write each username as a new row

print("CSV file created: not_following_back.csv")

In [ ]:
from google.colab import files

# Download the generated CSV file to the local machine (Colab-specific)
files.download("not_following_back.csv")

In [ ]:
import csv
import time
import os
import random

# Create the "pending_to_unfollow.csv" file from the original list (only once)
if not os.path.exists("pending_to_unfollow.csv"):
    with open("not_following_back.csv", "r", encoding="utf-8") as infile, \
         open("pending_to_unfollow.csv", "w", newline='', encoding="utf-8") as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        next(reader)  # Skip the header
        writer.writerow(["Username"])
        for row in reader:
            if row:
                writer.writerow(row)

# Load the list of remaining users to unfollow
usernames = []
with open("pending_to_unfollow.csv", "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header
    for row in reader:
        if row:
            usernames.append(row[0])

# Perform batch unfollowing (e.g., 80 users per run)
MAX_UNFOLLOWS = 80
unfollowed = []

for username in usernames[:MAX_UNFOLLOWS]:
    try:
        user_id = cl.user_id_from_username(username)
        cl.user_unfollow(user_id)
        print(f"Unfollowed: {username}")
        unfollowed.append(username)
        time.sleep(random.uniform(0.5, 1.5))  # Random delay to mimic human behavior
    except Exception as e:
        print(f"Error unfollowing {username}: {e}")

# Save the remaining users for the next run
remaining = usernames[MAX_UNFOLLOWS:]
with open("pending_to_unfollow.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Username"])
    for u in remaining:
        writer.writerow([u])

print(f"\n Unfollowed {len(unfollowed)} users in this session")
print(f"Remaining users for next time: {len(remaining)}")